In [3]:
import numpy as np
import random
import pandas as pd
import pickle
import torch
from torch import nn

In [4]:
train = pd.read_pickle("../data/ml-1m-split/train.pkl")
test = pd.read_pickle("../data/ml-1m-split/test.pkl")
full = pd.read_pickle("../data/ml-1m-split/full.pkl")

num_users = len(full.groupby("user").size())
num_items = len(full.groupby("item").size())

In [5]:
train.sort_values(by="user", axis=0).reset_index(drop=True)

,user,item,item_id,rating,timestamp
0,0,1104,1193,5,978300760
1,0,1574,1721,4,978300055
2,0,1421,1545,4,978824139
3,0,708,745,3,978824268
4,0,2969,3186,4,978300019
...,...,...,...,...,...
800188,6039,2317,2512,5,956715383
800189,6039,2952,3168,4,964828900
800190,6039,2309,2503,5,956704191
800191,6039,3272,3505,4,956704056


In [51]:
train_u = train.sort_values(by="user", axis=0).reset_index(drop=True).drop(["item_id", "timestamp"], axis=1).to_numpy()
train_m = train.sort_values(by="item", axis=0).reset_index(drop=True).drop(["item_id", "timestamp"], axis=1).to_numpy()
U_freqs = train.groupby("user").size().values
M_group = train.groupby("item").size()
M_freqs = np.zeros(num_items, dtype="int")
for i in M_group.index:
  M_freqs[i] = M_group[i]
  
U_start = np.cumsum(U_freqs)
M_start = np.cumsum(M_freqs)
U_start = np.insert(U_start, 0, 0)
M_start = np.insert(M_start, 0, 0)

In [52]:
len(M_start)

3707

In [7]:
U_freqs

array([ 42, 103,  41, ...,  16,  98, 273])

In [8]:
U_start[9]

639

In [79]:
def get_u_step(user):
    return np.mean(np.apply_along_axis(lambda x: (x[2] - np.dot(U[user[0]], M[x[1]])) * M[x[1]], 1, train_u[U_start[user[0]] : U_start[user[0] + 1]]), axis=0)

def get_m_step(item):
    return np.mean(np.apply_along_axis(lambda x: (x[2] - np.dot(U[x[0]], M[item[0]])) * U[x[0]], 1, train_m[M_start[item[0]] : M_start[item[0] + 1]]), axis=0)
     

In [97]:
%%time
num_factors = 30
lrate = 0.1
U = np.random.uniform(0, 0.6, (num_users, num_factors))
M = np.random.uniform(0, 0.6, (num_items, num_factors))

lrate * np.apply_along_axis(get_u_step, 1, np.arange(num_users).reshape(-1, 1))

CPU times: user 7.8 s, sys: 81 ms, total: 7.88 s
Wall time: 7.89 s


array([[0.05163281, 0.05683624, 0.04578771, ..., 0.05079945, 0.04549282,
        0.05502963],
       [0.04091203, 0.03370367, 0.03911205, ..., 0.04094239, 0.04413883,
        0.03814387],
       [0.04337609, 0.04666317, 0.04887803, ..., 0.04839563, 0.05177086,
        0.05284117],
       ...,
       [0.028283  , 0.02849512, 0.03034026, ..., 0.03234908, 0.02590766,
        0.02952698],
       [0.02714287, 0.03094314, 0.02730581, ..., 0.0299673 , 0.02974747,
        0.02918572],
       [0.01292772, 0.01016208, 0.01062782, ..., 0.00978245, 0.01192125,
        0.00985399]])

In [98]:
%%time
num_factors = 30
lrate = 0.1
U = np.random.uniform(0, 0.6, (num_users, num_factors))
M = np.random.uniform(0, 0.6, (num_items, num_factors))


for user in range(num_users):
  step = get_u_step([user])
  U[user] += lrate * step
  # step_size += np.sum(lrate * np.abs(step))

CPU times: user 7.5 s, sys: 75.7 ms, total: 7.57 s
Wall time: 7.58 s


In [69]:
def alt_min(num_factors, lrate):
    U = np.random.uniform(0, 0.6, (num_users, num_factors))
    M = np.random.uniform(0, 0.6, (num_items, num_factors))
    
    def evaluate():
      return np.sqrt(np.mean(np.apply_along_axis(lambda x: (x[2] - np.dot(U[x[0]], M[x[1]])) ** 2, 1, train_u), axis=0))
    
    rmse = evaluate()
    prev_rmse = 0
    outer = 0
    
    print("Initial RMSE: {}\n".format(rmse))
    
    def get_u_step(user):
      return np.mean(np.apply_along_axis(lambda x: (x[2] - np.dot(U[user], M[x[1]])) * M[x[1]], 1, train_u[U_start[user] : U_start[user + 1]]), axis=0)

    def get_m_step(item):
      return np.mean(np.apply_along_axis(lambda x: (x[2] - np.dot(U[x[0]], M[item])) * U[x[0]], 1, train_m[M_start[item] : M_start[item + 1]]), axis=0)
     
    print(np.apply_along_axis(get_u_step, 0, np.arange(10).reshape(-1, 1)))
    
    while abs(rmse - prev_rmse) > 0.001:
        prev_rmse = rmse
        
        # Optmize U
        step_size = 100
        inner = 0
        while abs(step_size) > 0.001:
            step_size = 0
            step = np.apply_along_axis(get_u_step, 1, np.arange(num_users).reshape(-1, 1))
            for user in range(num_users):
              step = get_u_step(user)
              U[user] += lrate * step
              step_size += np.sum(lrate * np.abs(step))
            
            step_size /= (num_users * num_factors)
            
            print("USER ITER {}\nStep size: {}\nRMSE: {}\n".format(inner, step_size, evaluate()))
            inner += 1
            
        
        # Optimize M
        step_size = 100
        inner = 0
        while abs(step_size) > 0.001:            
            step_size = 0
            for item in M_group.index:
              step = get_m_step(item)
              M[item] += lrate * step
              step_size += np.sum(lrate * np.abs(step))
            
            step_size /= (num_items * num_factors)
            
            print("ITEM ITER {}\nStep size: {}\nRMSE: {}\n".format(inner, step_size, evaluate()))
            inner += 1

        rmse = evaluate()

        print("\n============ ROUND {} ============\nRMSE: {}\nPrev RMSE: {}\nDiff: {}\n".format(outer, rmse, prev_rmse, rmse - prev_rmse))
        outer += 1

In [71]:
alt_min(30, 0.1)

Initial RMSE: 1.4834852036469584

USER ITER 0
Step size: 0.028978692307876676
RMSE: 1.3187450694340364

USER ITER 1
Step size: 0.020955963266334598
RMSE: 1.2232427796753311

USER ITER 2
Step size: 0.015156301275355956
RMSE: 1.1697227292384458

USER ITER 3
Step size: 0.010963153448213796
RMSE: 1.1403065283785454

USER ITER 4
Step size: 0.007931118156576458
RMSE: 1.1242198497168923

USER ITER 5
Step size: 0.005738391014332326
RMSE: 1.1153376546560545

USER ITER 6
Step size: 0.004152435920448024
RMSE: 1.1102998790210836

USER ITER 7
Step size: 0.0030051997190769794
RMSE: 1.1073002575011208

USER ITER 8
Step size: 0.0021752148178198604
RMSE: 1.105378457332423

USER ITER 9
Step size: 0.0015746737877991377
RMSE: 1.1040268641194038

USER ITER 10
Step size: 0.0011400934275439855
RMSE: 1.1029779548990855

USER ITER 11
Step size: 0.0008255705591080819
RMSE: 1.102090936040322

ITEM ITER 0
Step size: -0.007376753256799348
RMSE: 0.9816164385656844

ITEM ITER 1
Step size: -0.004447873374380763
RMSE:

KeyboardInterrupt: 

In [59]:
num_factors = 30
epochs = 1

def alt_min(num_factors, learning_rate):
  model = MatrixFactorization(num_users, num_items, num_factors)
  params = list(model.parameters())
  user_optimizer = torch.optim.SGD([params[0]], lr=learning_rate)
  item_optimizer = torch.optim.SGD([params[1]], lr=learning_rate)
  loss_fn = nn.MSELoss()

  for i in range(epochs):
    for (idx, [user, item, _, rating, _]) in train.iterrows():
      prediction = model(torch.LongTensor([user]), torch.LongTensor([item]))
      loss = loss_fn(prediction, torch.FloatTensor([rating]))
      grad = loss.backward()
      print(grad)
      user_optimizer.step()
      model.zero_grad()

In [25]:
avg_item_ratings = np.zeros(num_items)
for (item, df) in train.groupby("item"):
  avg_item_ratings[item] = np.mean(df["rating"].values)

In [33]:
train_shuffled = train.sample(frac=1).reset_index(drop=True)

In [36]:
len(train_shuffled)

800193

In [58]:
class MatrixFactorization(nn.Module):
  def __init__(self, num_users, num_items, num_factors):
    super().__init__()
    self.user_factors = nn.Embedding(num_users, num_factors, sparse=False)
    self.item_factors = nn.Embedding(num_items, num_factors, sparse=False)
    
    self.user_factors.weight.data.uniform_(-0.25, 0.25)
    self.item_factors.weight.data.uniform_(-0.25, 0.25)
    
  def forward(self, users, items):
    return torch.diagonal(torch.mm(self.user_factors(users), torch.transpose(self.item_factors(items), 0, 1)))

In [40]:
epochs = 10

def alt_min(num_factors, lrate):
  M = np.random.uniform(0, 0.25, (num_items, num_factors))
  # M[:, 0] = avg_item_ratings
  U = np.ones((num_users, num_factors))
  U_prev = np.zeros((num_users, num_factors))
  U_freqs = train.groupby("user").size().values.reshape(-1, 1)    
    
  for i in range(epochs):
    U_step = np.zeros((num_users, num_factors))
    total_loss = 0
    for (idx, [user, item, _, rating, _]) in train_shuffled.iterrows():
      residual = rating - np.dot(U[user], M[item])
      total_loss += residual ** 2
      U_step[user] += residual * M[item]
    
    U_step = lrate * (U_step / U_freqs)
    U += U_step
    print("EPOCH 1\nStep L2 Norm: {}\nRMSE: {}".format(np.linalg.norm(U_step), np.sqrt(total_loss / len(train_shuffled))))
    
          
      
    
  
  

In [43]:
alt_min(30, 0.5)

0
200000
400000
600000
800000
EPOCH 1
Step L2 Norm: 12.253237942641325
RMSE: 1.2000548624106067
0
200000
400000
600000
800000
EPOCH 1
Step L2 Norm: 9.43106566112119
RMSE: 1.1570828266019815
0
200000
400000
600000
800000
EPOCH 1
Step L2 Norm: 7.318747459406452
RMSE: 1.1311688881735902
0
200000
400000
600000
800000
EPOCH 1
Step L2 Norm: 5.752674947704704
RMSE: 1.1156156609148236
0
200000
400000
600000
800000
EPOCH 1
Step L2 Norm: 4.6083465600404665
RMSE: 1.1062456775370202
0
200000


KeyboardInterrupt: 